In [ ]:
import torch
import torch.nn as nn

from torch_bwim.dataset.ToTensorAdapter import ToTensorAdapter
from torch_bwim.dataset.TorchDataUtils import TorchDataUtils
from torch_bwim.dataset.TrainDictDataset import TrainDictDataset
from torch_bwim.lr_schedulers.modules.CosAnnealingScheduler import CosAnnealingScheduler
from torch_bwim.nets.ActivationFunctions import ActivationFunctions
from torch_bwim.nets.NetBase import NetBase
from torch_bwim.optimizers.modules.AdamFactory import AdamFactory
from torch_bwim.trainers.modules.StandardNetTrainer import StandardNetTrainer

%load_ext autoreload
%autoreload 2

In [ ]:
class VariableLengthInputOutputNet(NetBase):

    class Config(object):
        def __init__(self, neuron_count1, neuron_count2):
            super().__init__()
            self.neuron_count1 = neuron_count1
            self.neuron_count2 = neuron_count2

    def __init__(self, config: Config):
        super().__init__(config)
        self.config = config
        self.sin_layers = self._create_layers()
        self.cos_layers = self._create_layers()

    def _create_layers(self):
        cfg = self.config
        return nn.Sequential(
            nn.Linear(1, cfg.neuron_count1),
            ActivationFunctions.get_function(ActivationFunctions.Types.LeakyReLU),
            nn.Linear(cfg.neuron_count1, cfg.neuron_count2),
            ActivationFunctions.get_function(ActivationFunctions.Types.LeakyReLU),
            nn.Linear(cfg.neuron_count2, 1),
            ActivationFunctions.get_function(ActivationFunctions.Types.LeakyReLU)
        )

    def forward(self, input1, input2):
        orig_size = input1.shape

        reshape_size = (orig_size[0] * orig_size[1], 1)
        input1 = torch.reshape(input1, reshape_size)
        input2 = torch.reshape(input2, reshape_size)
        out1 = self.sin_layers.forward(input1)
        out2 = self.cos_layers.forward(input2)

        output = out1 + out2
        output = torch.reshape(output, orig_size)
        return output

In [ ]:
example_net = VariableLengthInputOutputNet(
    config=VariableLengthInputOutputNet.Config(
        neuron_count1=8, neuron_count2=16
    )
)

output = example_net.forward(torch.empty(8, 16), torch.empty(8, 16))
print(f'Output size: {output.size()}')
output, = example_net(torch.empty(8, 16), torch.empty(8, 16))
TorchDataUtils.check_shape(output, expected_shape=(8, 16))

In [ ]:
import numpy as np


def noise(size):
    return np.random.normal(0.0, 0.01, size)

INPUT_FEATURE1 = 'input1'
INPUT_FEATURE2 = 'input2'
LABEL_FEATURE = 'label'
SIZE_OF_INPUT = 'size_of_input'

def generate_dataset(variable_input_size=False):
    generated_dataset = []
    PI = 3.14
    for i in range(2048):
        size = ((i // 512) + 1) * 16 if variable_input_size else 32
        new_data = dict()
        new_data[SIZE_OF_INPUT] = size
        new_data[INPUT_FEATURE1] = input1 = np.random.uniform(-PI, +PI, size) + noise(size)
        new_data[INPUT_FEATURE2] = input2 = np.random.uniform(-PI, +PI, size) + noise(size)

        new_data[LABEL_FEATURE] = np.sin(input1) + np.cos(input2) + noise(size)
        generated_dataset.append(new_data)
    return generated_dataset

dict_dataset = generate_dataset()

In [ ]:
class ExampleInputToTensor(ToTensorAdapter):
    def __init__(self):
        super().__init__(length_out=2)

    def process(self, data: dict):
        in1 = torch.from_numpy(data[INPUT_FEATURE1].astype(np.float32))
        in2 = torch.from_numpy(data[INPUT_FEATURE2].astype(np.float32))
        return in1, in2


class ExampleLabelToTensor(ToTensorAdapter):
    def __init__(self):
        super().__init__(length_out=1)

    def process(self, data: dict):
        label = torch.from_numpy(data[LABEL_FEATURE].astype(np.float32))
        return label,


In [ ]:
dataset = TrainDictDataset(
    dict_dataset=dict_dataset,
    input_to_tensor_adapter=ExampleInputToTensor(),
    label_to_tensor_adapter=ExampleLabelToTensor()
)

In [ ]:
index, input1, input2, label = dataset[3]
print(index)

original_dict = dataset.get_sample(index)
print(original_dict is dict_dataset[3])

In [ ]:
trainer = StandardNetTrainer(
    train_config=StandardNetTrainer.Config(
        batch_size=256, shuffle_dataset=True,
        random_state=42
    ),
    logger=print
)

net = VariableLengthInputOutputNet(
    config=VariableLengthInputOutputNet.Config(
        neuron_count1=16, neuron_count2=64
    )
)

train_dataset, val_dataset = TorchDataUtils.split_dataset(dataset, length_ratios=[0.8, 0.2])

trainer.initialize(
    net=net,
    train_dataset=train_dataset, val_dataset=val_dataset,
    dataset_provider=dataset,
    loss_function=nn.MSELoss(),
    scheduler_config=CosAnnealingScheduler.Config(
        step_period=128, annealing_period_in_steps=256, lr_ratio=0.5
    ),
    optimizer_config=AdamFactory.Config(learning_rate=1e-3, weight_decay=1e-6, amsgrad=True),
    cuda=True,
)
trainer.train(epoch_num=300)